In [1]:
import pandas as pd
#import boto3
#import anthropic_bedrock
#from anthropic_bedrock import AnthropicBedrock
from tqdm import tqdm
import hashlib
import argparse
import csv
import re
import json
import configparser
import os
import numpy as np
from vllm import LLM, SamplingParams

from tqdm import tqdm
tqdm.pandas()

In [2]:
dataset = pd.read_csv('all_qac_triplets.csv')[['original_ques','context']]
dataset.rename(columns = {'original_ques':'generated_question'}, inplace = True)
dataset["QID"] = pd.factorize(dataset["generated_question"])[0].astype(str)

dataset

,generated_question,context,QID
0,What type of ECG will be performed at screening?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,0
1,How often will the ECG be performed?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,1
2,What procedures will be used to perform the ECG?,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,2
3,What should be done with clinically significan...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,3
4,Where should clinically significant abnormal E...,7.1.2.4 Lead Electrocardiogram (ECG)\nA standa...,4
...,...,...,...
723815,What is the title of the clinical trial protocol?,Title: Preclinical Development of Inducible My...,278476
723816,What is being developed preclinically?,Title: Preclinical Development of Inducible My...,457655
723817,What is the intended target of the GoCAR-NK ce...,Title: Preclinical Development of Inducible My...,457656
723818,"What section is indicated by the word ""Abstract""?",Title: Preclinical Development of Inducible My...,453154


In [3]:
question_id = dataset.drop_duplicates(subset=['generated_question','QID'])[['generated_question','QID']].reset_index()
question_id

,index,generated_question,QID
0,0,What type of ECG will be performed at screening?,0
1,1,How often will the ECG be performed?,1
2,2,What procedures will be used to perform the ECG?,2
3,3,What should be done with clinically significan...,3
4,4,Where should clinically significant abnormal E...,4
...,...,...,...
457653,723810,"What section of the protocol is labeled ""Abstr...",457652
457654,723813,What are the biomarkers being associated with ...,457653
457655,723814,What is the name and number of the clinical tr...,457654
457656,723816,What is being developed preclinically?,457655


In [4]:
len(question_id)

457658

In [115]:
def prompt1(alltext):
    return f'''
I'm going to give you a question generated from a clinical trial protocol document. Then I'm going to ask you to generate an equivalence class of that question with four rephrased questions, ranging from highly syntactically similar to less syntactically similar. The meaning of the questions should remain the same. Here is the question:

<question>

{alltext}

</question>

Generate four rephrased versions of the question. Make sure you rephrase only the question given within the tags. Generate exactly four rephrased questions excluding the original question with no exceptions.

Each question and its rephrased versions should follow this format:
Q1: Original question.
Q1.1: First rephrased question (high syntactical similarity).
Q1.2: Second rephrased question.
Q1.3: Third rephrased question.
Q1.4: Fourth rephrased question (low syntactical similarity).

The format of your overall response should match what's shown between the tags. Ensure to follow the formatting and spacing exactly.

<example>

Q1: What safety measures are in place for participants?
Q1.1: What safety protocols are in place for participants?
Q1.2: What safety procedures are implemented for participants?
Q1.3: What actions are taken to ensure participant safety?
Q1.4: How is the safety of participants ensured?

Q2: How long is the duration of the study?
Q2.1: What is the duration of the study?
Q2.2: What is the study's duration?
Q2.3: How many months does the study last?
Q2.4: What is the total time frame for the study?

Q3: What are the inclusion criteria for participants?
Q3.1: What criteria must participants meet to be included?
Q3.2: What are the requirements for participant inclusion?
Q3.3: Who can be included in this study?
Q3.4: What conditions must participants fulfill to join the study?

</example>

You MUST Provide exactly four rephrased questions for every given question. Don't skip any rephrased question. Ensure that only the given question and its rephrased versions are included in the output in the format shown in the example. The output should strictly follow the provided format, and the rephrased questions must not deviate significantly from the original question. The first rephrased question should be highly similar to the original question and the similarity should decrease gradually. Don't include any extra information in the output apart from the original questions and the rephrased questions excactly in the format provided .
'''


In [116]:
def system_prompt(prompt):
    #https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
    return f'''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant. Your carefully follow instructions.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''

In [ ]:
prompts=[]
for i,row in question_id.iterrows():
    if(i not in not_four_new):
        continue
    
    text=row['generated_question']
    prompt=system_prompt(prompt1(text))
    prompts.append(prompt)

len(prompts)    

In [119]:
llm = LLM(model="/lockbox/models/Meta-Llama-3-8B-Instruct", gpu_memory_utilization = 0.8,enforce_eager=True)
sampling_params = SamplingParams(temperature=0.05, top_p=0.95, top_k = 40, max_tokens = 2048,)
out=[]
try:
    generated_outputs = llm.generate(prompts, sampling_params)
    for output in generated_outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        out.append(generated_text) 
except Exception:
            print(Exception)  


INFO 07-27 18:20:55 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/lockbox/models/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='/lockbox/models/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/lockbox/models/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-27 18:21:24 model_runner.py:160] Loading model weights took 14.9575 GB
INFO 07-27 18:21:25 gpu_executor.py:83] # GPU blocks: 23966, # CPU blocks: 2048


Processed prompts: 100%|██████████| 266/266 [00:17<00:00, 15.12it/s, est. speed input: 8852.17 toks/s, output: 699.40 toks/s] 


In [42]:
def extract_rephrased_questions(output):
    # Extract the rephrased questions using regex, excluding the original question
    rephrased_questions_matches = re.findall(r"Q\d+\.\d+:\s*(.*?)\s*(?=Q\d|\Z)", output, re.DOTALL)
    rephrased_questions = [match.strip() for match in rephrased_questions_matches]
    return rephrased_questions


extract_rephrased_questions(out[4])


['What does the SAC provide input on?',
 'What information does the SAC provide input on?',
 'What type of information does the SAC offer input on?',
 'What kind of guidance does the SAC offer in terms of decision-making?']

In [70]:
final_json={}
for i,row in question_id.iterrows():
    
    id=row['QID']
    regenrated_questions=extract_rephrased_questions(out_new[i])
    temp={'original_question':row['generated_question'],'rephrased':regenrated_questions}
    final_json[id]=temp

len(final_json)    

457658

In [7]:
keys = list(final_json.keys())  # Create a list of keys to iterate over

keys_to_delete = []  # List to hold keys to delete

for key in keys:
    val = final_json[key]['rephrased']
    if len(val) != 4:
        keys_to_delete.append(key)  # Add key to delete list if condition is met

for key in keys_to_delete:
    del final_json[key]  # Delete keys outside the iteration


In [8]:
with open('EquivalenceClass_cleaned.json','w') as f:
    json.dump(final_json ,f)

In [6]:
with open('EquivalenceClass.json','r') as f:
    final_json=json.load(f)

to_show={k: final_json[k] for k in list(final_json.keys())[:10]} 
to_show

{'0': {'original_question': 'What type of ECG will be performed at screening?',
  'rephrased': ['What type of electrocardiogram will be conducted at the screening stage?',
   'What kind of ECG test will be done during the screening process?',
   'What is the nature of the ECG examination that will take place at the initial assessment?',
   'What is the purpose of the ECG test that will be performed at the beginning of the study?']},
 '1': {'original_question': 'How often will the ECG be performed?',
  'rephrased': ['How frequently will the ECG be conducted?',
   'How many times will the ECG be done during the study?',
   'What is the schedule for ECG testing?',
   'At what intervals will the ECG be recorded throughout the study?']},
 '2': {'original_question': 'What procedures will be used to perform the ECG?',
  'rephrased': ['What procedures will be employed to conduct the ECG?',
   'How will the ECG be performed?',
   'What methods will be used to record the ECG?',
   'What techniqu